### ECE/CS/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2022 ###

# Optimal Scientific Victory Path for Civ 6 Players #

## Team Optimus Prime ##

#### Prajwal Dinesh Bhagwat (pbhagwat@wisc.edu), Cameron Boeder (cboeder@wisc.edu), Lingyun Xiao (lingyun.xiao@wisc.edu), and Aksel Balkir (abalkir@wisc.edu)

*****

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Data](#3.-Data)
1. [Code Implementation](#4.-Code-Implementation)
1. [Results and Discussion](#5.-Results-and-discussion)
1. [Conclusion](#6.-Conclusion)

## 1. Introduction ##

Civilization 6 (Civ6) is a popular strategic game where each player's goal is to achieve victory through various paths (domination, scientific, diplomatic, etc.) Replicating every single detail in the game is indeed challenging given the scale of Civ6. Therefore, our group focuses on the scientific victory path and finds the optimal path through the technology tree. 

In our baseline model, each technology in the tree costs ``science`` to research, and the player receives a certain amount of ``science`` per turn which they can contribute to researching one technology at a time. The tech tree represents the prerequisites for each technology. In other words, in order to research a technology the player must have researched all parent technologies in the tech tree. Also, players may choose to use ``production`` to construct research buildings to generate more ``science`` per turn, and each research building is unlocked by certain technologies.

For players to win a science victory, they must launch: a satellite, a moon landing, and a mars colony - each of which has an associated technology to research in the tech tree. Thus, the objective of this optimization problem is to find the optimal path through the tech tree that allows the players to complete research for and launch each of the victory requirements at the earliest possible turn. By transforming a rather complex game setting to an optimization model, our group hopes to offer players a new perspective on improving the chance of achieving a scientific victory. 

The project report is organized as follows: Section 2 formalizes the baseline mathematical model as well as the alternative model with more variables and assumptions closer to the real game. Section 3 explains what the necessary data is and how it was obtained. Section 4 presents the solution to each model. Section 5 demonstrates the results from the optimization model and their implications. Finally, Section 5 displays the results and discusses further improvements that could be made in the future to better formulate the game.

The images below show the techonology tree that our optimization problem focuses on.

## 2. Mathematical model ##

### Decision variables

A decision variable to track the technology research 

$$w[i,j] \in \{1,0\}$$
$$i \in \text{set of technologies}$$
$$j \in \text{set of turns}$$

(i.e. $w[3,7]$ is 1 if technology 3 is being researched on turn 7, and 0 if technology 3 was researched on another turn)

A decision variable to maintain the status of researching the technology for all turns

$$x[i,j] \in \{1,0\}$$
$$i \in \text{set of technologies}$$
$$j \in \text{set of turns}$$

(i.e. $x[3,7]$ is 1 if technology 3 has been researched before turn 7, and 0 if technology 3 has not been researched before turn 7)

A decision variable to to track the buildings built 

$$y[i,j] \in \{1,0\}$$
$$i \in \text{set of buildings}$$
$$j \in \text{set of turns}$$

(i.e. $y[3,7]$ is 1 if building 3 is being constructed on turn 7, and 0 if building 3 was constructed on another turn)

A decision varibale to maintain the status of buildings built for all the turns

$$z[i,j] \in \{1,0\}$$
$$i \in \text{set of buildings}$$
$$j \in \text{set of turns}$$

(i.e. $z[3,7]$ is 1 if building 3 has been constructed before turn 7, and 0 if building 3 has not been constructed before turn 7)

### Constraints 

A constraint which limits the tech researched to one tech per turn 

$$ \sum_{i=1}^{\text{num of tech}} w[i,j] = 1$$
$$ j \in \text{set of turns}$$

Each tech is only researched once

$$ \sum_{j=1}^{\text{num of turns}} w[i,j] = 1$$
$$ i \in \text{set of technology}$$

Each tech can only be researched after its predecessors techs have be researched
$$ j \text{ of } w[i,j] \geq j \text{ of } w[k,j] \text{ predecessor } $$
$$ i \in \text{set of technologies}$$
$$ k \in \text{set of predecessor technologies}$$

The number of techs researched equals the number of turns passed

$$ \sum_{i=1}^{\text{num of tech}} x[i,j] = j$$
$$ j \in \text{set of turns}$$

For each $x$ element if the corresponding tech is researched then tech status has to be set to 1

$$ x[i,j] \geq w[i,j] $$
$$i \in \text{set of technologies}$$
$$j \in \text{set of turns}$$

The status of each tech stays on for every turn after it has been researched

$$ x[i,j+1] \geq x[i,j] $$
$$i \in \text{set of technologies}$$
$$j \in \text{set of turns}$$

If the successer status is one then the predesessor status has to be one 

$$ \text{sum of one to } j \text{ of } x[i,j] \geq \text{ sum of one to }j \text{ of } x[k,j] \text{ predecessor } $$
$$ i \in \text{set of technologies}$$
$$ k \in \text{set of predecessor technologies}$$

A constraint which limits the buildings constructed to at most one tech per turn 

$$ \sum_{i=1}^{\text{num of buildings}} y[i,j] \leq 1$$
$$ j \in \text{set of turns}$$

Each building is only constructed once

$$ \sum_{j=1}^{\text{num of turns}} y[i,j] = 1$$
$$ i \in \text{set of buildings}$$

The predecessor tech has to be completed before the buildings 

$$ j \text{ of } y[i,j] \geq j \text{ of } w[k,j] \text{ predecessor } $$
$$ i \in \text{set of buildings}$$
$$ k \in \text{set of predecessor technologies}$$

For each $y$ element if the corresponding building is constructed then building status has to be set to 1

$$ z[i,j] \geq y[i,j] $$
$$i \in \text{set of buildings}$$
$$j \in \text{set of turns}$$

The status of each building stays on for every turn after it has been researched 

$$ z[i,j+1] \geq z[i,j] $$
$$i \in \text{set of buildings}$$
$$j \in \text{set of turns}$$

If the successer status is one then the predesessor status has to be one. The buildings have tech as their predecessors. 

$$ \text{sum of one to } j \text{ of } z[i,j] \geq \text{ sum of one to }j \text{ of } x[k,j] \text{ predecessor } $$
$$ i \in \text{set of technologies}$$
$$ k \in \text{set of predecessor technologies}$$


### Objective function

The goal of this project is to minimize the turns necessary to research the three technologies for a science victory. This would be calculated by dividing the cost of the technologies by the science per turn for each technology, but dividing by a decision variable is difficult for the optimizer to solve. Instead we have replaced this objective with the maximization of the science per turn scaled by the science cost of each technology. The science cost of each tech can be found in our CSV data frame and are constant. The science per turn is calculated by the building status of each building multiplied by the science produced for each building. This method exploits min to max conversion and the fact that from our first model we found that not all the techs must be researched at the end. 

$$ \max \quad (\alpha[1]z[1,:] w+\alpha[2]z[2,:]w+\alpha[3]z[3,:] w + 2 ) *(\text{science cost for each tech})$$

Here $\alpha$ is the science earned for each building. 2 is added to this objective because we are earning a baseline of 2 science per turn. Science cost for each tech is a given data. 


## 3. Data ##

The data set used in our analysis could be found easily by web scraping on the official fandom webpage of Civilization 6 (https://civilization.fandom.com/). 

Below is a python script for webscraping the data we used(Note, this will not run in a Julia notebook):

In [ ]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen
import re 

In [ ]:
url='https://civilization.fandom.com/wiki/List_of_technologies_in_Civ6'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')
tables = soup.find_all('table')

In [ ]:
header = ['Technology', 'Prerequisites', 'Eureka', 'Infrastructure', 'Units', 'Effects']

tech = []
pre = []
eureka = []
inf = []
unit = []
effect = []

for table in tables:
  rows = table.find_all('tr')
  ths = table.find_all('th')
  headings = [th.text.strip() for th in ths]
  for row in rows:
    cells = row.find_all('td')
    if headings == header:
      if len(cells) > 1:
        techs = cells[0]
        tech.append(techs.text.strip())
            
        pres = cells[1]
        pre.append(pres.text.strip())

        eurekas = cells[2]
        eureka.append(eurekas.text.strip())

        infs = cells[3]
        inf.append(infs.text.strip())

        units = cells[4]
        unit.append(units.text.strip())

        effects = cells[5]
        effect.append(effects.text.strip())
            
data = np.row_stack([header[:2], np.column_stack([tech, pre])])

In [ ]:
sci_costs = ['Science Cost']

for techs in data[1:-1,0]:
  tech = re.sub('[^a-zA-Z]+', '', techs)
  tech = re.sub("([a-z])([A-Z])", "\\1_\\2", tech)
  url = 'https://civilization.fandom.com/wiki/'+str(tech)+'_(Civ6)'
  html = urlopen(url) 
  soup = BeautifulSoup(html, 'html.parser')
  divs = soup.find_all('div')
  cost = 0
  for div in divs:
    if 'data-source' in div.attrs:
      if div['data-source'] == 'cost':
        cost = div
  if cost == 0:
    sci_cost = 'NA'
  else:
    sci_cost = cost.text.strip()
    sci_cost = sci_cost[sci_cost.find('\n')+1:]
  sci_costs.append(sci_cost)

In [ ]:
tech_data = np.column_stack([data[:-1,:], sci_costs])
pd.DataFrame(tech_data).to_csv("Civ6_Techs.csv", header=None, index=None)

In [ ]:
url='https://civilization.fandom.com/wiki/List_of_buildings_in_Civ6'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')
tables = soup.find_all('table')

In [ ]:
header1 = ['Building', 'District', 'Unlocked with', 'Era']
header2 = ['Building', 'Prerequisites'] 

building = []
district = []
pre = []
era = []

for table in tables:
  rows = table.find_all('tr')
  ths = table.find_all('th')
  headings = [th.text.strip() for th in ths]
  for row in rows:
    cells = row.find_all('td')
    if headings == header1:
      if len(cells) > 2:
        buildings = cells[0]
        building.append(buildings.text.strip())
            
        districts = cells[1]
        district.append(districts.text.strip())

        pres = cells[2]
        pre.append(pres.text.strip())

            
data = np.row_stack([header2, np.column_stack([building, pre])])

In [ ]:
pro_costs = ['Production Cost']

for buildings in data[1:,0]:
  building = re.sub("\(.*?\)","()", buildings)
  building = re.sub('[^a-zA-Z]+', '', building)
  building = re.sub("([a-z])([A-Z])", "\\1_\\2", building)
  url = 'https://civilization.fandom.com/wiki/'+str(building)+'_(Civ6)'
  html = urlopen(url) 
  soup = BeautifulSoup(html, 'html.parser')
  divs = soup.find_all('div')
  cost = 0
  for div in divs:
    if 'data-source' in div.attrs:
      if div['data-source'] == 'cost':
        cost = div
  if cost == 0:
    pro_cost = 'NA'
  else:
    pro_cost = cost.text.strip()
    pro_cost = pro_cost.split(' ',1)[0]
  pro_costs.append(pro_cost)

In [ ]:
sci_boosts = ['Science Boost']

for buildings in data[1:,0]:
  building = re.sub("\(.*?\)","()", buildings)
  building = re.sub('[^a-zA-Z]+', '', building)
  building = re.sub("([a-z])([A-Z])", "\\1_\\2", building)
  url = 'https://civilization.fandom.com/wiki/'+str(building)+'_(Civ6)'
  html = urlopen(url) 
  soup = BeautifulSoup(html, 'html.parser')
  divs = soup.find_all('div')
  for div in divs:
    if 'data-source' in div.attrs:
      if div['data-source'] == 'effect':
        effects = div
  sci_boost = effects.text.strip()
  if sci_boost.find('Science') > 0:
    sci_boost = sci_boost[:sci_boost.find('Science')][::-1]
    sci_boost = sci_boost[:sci_boost.find('+')]
    sci_boost = sci_boost.replace(' ', '')
    if sci_boost.isnumeric() == False:
      sci_boost = 'NA'
  else:
    sci_boost = 'NA'
  sci_boosts.append(sci_boost)

In [ ]:
pro_boosts = ['Production Boost']

for buildings in data[1:,0]:
  building = re.sub("\(.*?\)","()", buildings)
  building = re.sub('[^a-zA-Z]+', '', building)
  building = re.sub("([a-z])([A-Z])", "\\1_\\2", building)
  url = 'https://civilization.fandom.com/wiki/'+str(building)+'_(Civ6)'
  html = urlopen(url) 
  soup = BeautifulSoup(html, 'html.parser')
  divs = soup.find_all('div')
  for div in divs:
    if 'data-source' in div.attrs:
      if div['data-source'] == 'effect':
        effects = div
  pro_boost = effects.text.strip()
  if pro_boost.find('Production') > 0:
    pro_boost = pro_boost[:pro_boost.find('Production')][::-1]
    pro_boost = pro_boost[:pro_boost.find('+')]
    pro_boost = pro_boost.replace(' ', '')
    if pro_boost.isnumeric() == False:
      pro_boost = 'NA'
  else:
    pro_boost = 'NA'
  pro_boosts.append(pro_boost)

In [ ]:
building_data = np.column_stack([data, pro_costs, sci_boosts, pro_boosts])
pd.DataFrame(building_data).to_csv("Civ6_Buildings.csv", header=None, index=None)

In summary, The data we have includes:

#### 1. A csv of technologies with each row including: ####

    Name of technology
    Prerequisite technologies
    Cost in "science"
    
#### 2. A csv of science producing buildings with each row including: ####

    Name of building
    Prerequisite technologies
    Boost in “science” that each building provides
    

In [58]:
using CSV
using DataFrames

df_techs = CSV.read("Civ6_Techs.csv", DataFrame)
;
df_techs

,Technology,Prerequisites,Science Cost,Turns (+2 science per turn)
,String31,String,Int64,Int64
1,Pottery,NA,25,13
2,AnimalHusbandry,NA,25,13
3,Mining,NA,25,13
4,Sailing,NA,50,25
5,Astrology,NA,50,25
6,Irrigation,Pottery,50,25
7,Writing,Pottery,50,25
8,Archery,AnimalHusbandry,50,25
9,Masonry,Mining,80,40


In [59]:
using CSV
using DataFrames

df_buildings = CSV.read("Civ6_Buildings.csv", DataFrame)
;
df_buildings

,Building,Prerequisites,Production Cost,Science Boost,Production Boost
,String15,String31,String3,String3,String3
1,Palace,None,NA,2,2
2,Library,Writing,90,2,NA
3,Barracks,BronzeWorking,90,NA,1
4,WaterMill,Wheel,80,NA,1
5,Stable,HorsebackRiding,120,NA,1
6,Workshop,Apprenticeship,195,NA,2
7,Armory,MilitaryEngineering,195,NA,2
8,University,Education,250,4,NA
9,Factory,Industrialization,390,NA,3


## 4. Code Implementation ##

In [60]:
#Load the data

using CSV
using DataFrames

df_techs = CSV.read("Civ6_Techs.csv", DataFrame)
;

In [61]:
#Build the necessary dictionaries, matrices, and vectors corresponding to the tech data

techs = []
for i=1:length(df_techs[:,1])
    push!(techs, Symbol(df_techs[i,1]))
end
tech_dict = Dict(zip(techs, Array(1:length(techs))))
tech_dict_rev = Dict(zip(Array(1:length(techs)), techs))

tech_predecessors = []
for i=1:length(df_techs[:,1])
    if df_techs[i,2] == "NA"
        push!(tech_predecessors, [])
    elseif match(r".,.", df_techs[i,2]) !== nothing
        tech_preds = []
        commas = findall(",", df_techs[i,2])
        if length(commas) > 1
            for j=1:length(commas)
                if j == 1
                    push!(tech_preds, tech_dict[Symbol(SubString(df_techs[i,2], 1:commas[j][1]-1))])
                elseif j == length(findall(",", df_techs[i,2]))
                    push!(tech_preds, tech_dict[Symbol(SubString(df_techs[i,2], commas[j-1][1]+1:commas[j][1]-1))])
                    push!(tech_preds, tech_dict[Symbol(SubString(df_techs[i,2], commas[j][1]+1))])
                else
                    push!(tech_preds, tech_dict[Symbol(SubString(df_techs[i,2], commas[j-1][1]+1:commas[j][1]-1))])
                end     
            end
        else
            push!(tech_preds, tech_dict[Symbol(SubString(df_techs[i,2], 1:commas[1][1]-1))])
            push!(tech_preds, tech_dict[Symbol(SubString(df_techs[i,2], commas[1][1]+1))])
        end
        push!(tech_predecessors, Vector{Int}(tech_preds))
    else
        push!(tech_predecessors, [tech_dict[Symbol(df_techs[i,2])]])
    end
end
tech_pred_dict = Dict(zip(Array(1:length(techs)), tech_predecessors))

tech_sci_cost = df_techs[:,3]
;

In [62]:
#Model with no buildings

using JuMP, Gurobi

pos = Array(1:length(techs))'

m = Model(with_optimizer(Gurobi).Optimizer)

@variable(m, tech_turn[1:length(techs),1:length(techs)], Bin)

##Tech Turns - The turn each tech is researched
##Row = Tech
##Column = Turn

###Only one tech can be researched at a time
for j=1:length(techs)
    @constraint(m, sum(tech_turn[:,j]) == 1)
end

###Each tech is only researched once
for i=1:length(techs)
    @constraint(m, sum(tech_turn[i,:]) == 1)
end

###Each tech can only be researched after its predecessors techs have be researched
for i=1:length(techs)
    for j in tech_pred_dict[i]
        @constraint(m, pos*tech_turn[i,:] >= pos*tech_turn[j,:] + 1)
    end
end

@objective(m, Min, sum(pos.*tech_turn[tech_dict[:NuclearFusion],:]') + 
    sum(pos.*tech_turn[tech_dict[:Robotics],:]') + 
    sum(pos.*tech_turn[tech_dict[:Nanotechnology],:]'))

optimize!(m)
;

Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-04
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 230 rows, 4761 columns and 22218 nonzeros
Model fingerprint: 0xed530433
Variable types: 0 continuous, 4761 integer (4761 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+00, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve added 49 rows and 0 columns
Presolve removed 0 rows and 810 columns
Presolve time: 0.07s
Presolved: 279 rows, 3951 columns, 13496 nonzeros
Variable types: 0 continuous, 3951 integer (3902 binary)

Root relaxation: objective 1.153810e+02, 2887 iterations, 0.10 seconds (0.17 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0 

In [27]:
using NamedArrays

raw = broadcast(abs, Array(JuMP.value.(tech_turn)))

results = NamedArray(raw, (techs,pos'), ("Technology","Turn"))

63×63 Named Matrix{Float64}
   Technology ╲ Turn │   1    2    3    4    5  …   59   60   61   62   63
─────────────────────┼────────────────────────────────────────────────────
:Pottery             │ 0.0  0.0  1.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0
:AnimalHusbandry     │ 0.0  0.0  0.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0
:Mining              │ 1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
:Sailing             │ 0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
:Writing             │ 0.0  0.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0
:Archery             │ 0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
:Masonry             │ 0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
:BronzeWorking       │ 0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
:Wheel               │ 0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
:Currency            │ 0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
:HorsebackRiding     │ 0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0

In [21]:
#Load the data

using CSV
using DataFrames

df_techs = CSV.read("Civ6_Techs_NoExtras.csv", DataFrame)
df_buildings = CSV.read("Civ6_ScienceBuildings.csv", DataFrame)
;

In [22]:
#Build the necessary dictionaries, matrices, and vectors corresponding to the tech data

techs = []
for i=1:length(df_techs[:,1])
    push!(techs, Symbol(df_techs[i,1]))
end
tech_dict = Dict(zip(techs, Array(1:length(techs))))
tech_dict_rev = Dict(zip(Array(1:length(techs)), techs))

tech_predecessors = []
for i=1:length(df_techs[:,1])
    if df_techs[i,2] == "NA"
        push!(tech_predecessors, [])
    elseif match(r".,.", df_techs[i,2]) !== nothing
        tech_preds = []
        commas = findall(",", df_techs[i,2])
        if length(commas) > 1
            for j=1:length(commas)
                if j == 1
                    push!(tech_preds, tech_dict[Symbol(SubString(df_techs[i,2], 1:commas[j][1]-1))])
                elseif j == length(findall(",", df_techs[i,2]))
                    push!(tech_preds, tech_dict[Symbol(SubString(df_techs[i,2], commas[j-1][1]+1:commas[j][1]-1))])
                    push!(tech_preds, tech_dict[Symbol(SubString(df_techs[i,2], commas[j][1]+1))])
                else
                    push!(tech_preds, tech_dict[Symbol(SubString(df_techs[i,2], commas[j-1][1]+1:commas[j][1]-1))])
                end     
            end
        else
            push!(tech_preds, tech_dict[Symbol(SubString(df_techs[i,2], 1:commas[1][1]-1))])
            push!(tech_preds, tech_dict[Symbol(SubString(df_techs[i,2], commas[1][1]+1))])
        end
        push!(tech_predecessors, Vector{Int}(tech_preds))
    else
        push!(tech_predecessors, [tech_dict[Symbol(df_techs[i,2])]])
    end
end
tech_pred_dict = Dict(zip(Array(1:length(techs)), tech_predecessors))

tech_sci_cost = df_techs[:,3]
;

In [23]:
#Build the necessary dictionaries, matrices, and vectors corresponding to the building data

buildings = []
for i=1:length(df_buildings[:,1])
    push!(buildings, Symbol(df_buildings[i,1]))
end
building_dict = Dict(zip(buildings, Array(1:length(buildings))))

building_predecessors = []
for i=1:length(df_buildings[:,1])
    push!(building_predecessors, [tech_dict[Symbol(df_buildings[i,2])]])
end
building_pred_dict = Dict(zip(Array(1:length(buildings)), building_predecessors))

sci_per_building = df_buildings[:,3]'
;

In [48]:
#Model with buildings

using JuMP, Gurobi

pos = Array(1:length(techs))'

m = Model(with_optimizer(Gurobi).Optimizer)

@variable(m, final_tech_turn[1:length(techs),1:length(techs)], Bin)
@variable(m, tech_status[1:length(techs),1:length(techs)], Bin)

@variable(m, building_turn[1:length(buildings),1:length(techs)], Bin)
@variable(m, building_status[1:length(buildings),1:length(techs)], Bin)

##Tech Turns - The turn each tech is researched
##Row = Tech
##Column = Turn

###Only one tech can be researched at a time
for j=1:length(techs)
    @constraint(m, sum(final_tech_turn[:,j]) == 1)
end

###Each tech is only researched once
for i=1:length(techs)
    @constraint(m, sum(final_tech_turn[i,:]) == 1)
end

###Each tech can only be researched after its predecessors techs have be researched
for i=1:length(techs)
    for j in tech_pred_dict[i]
        @constraint(m, pos*final_tech_turn[i,:] >= pos*final_tech_turn[j,:] + 1)
    end
end

##Tech Status - The techss that are active each turn
##Row = Building
##Column = Turn

###The number of techs researched equals the number of turns passed.
for j=1:length(techs)
    @constraint(m, sum(tech_status[:,j]) == j)
end

###The status of each tech is on after it has been researched
for i=1:length(techs)
    for j=1:length(techs)
        @constraint(m, tech_status[i,j] >= final_tech_turn[i,j])
    end
end

###The status of each tech stays on for every turn after it has been researched
for i=1:length(techs)
    for j=1:(length(techs) - 1)
       @constraint(m, tech_status[i,j] <= tech_status[i,j+1])
    end
end

###Each tech can only be researched after its predecessor technologies are researched
for i=1:length(techs)
    for j in tech_pred_dict[i]
        @constraint(m, sum(tech_status[i,:]) <= sum(tech_status[j,:]))
    end
end

##Building Turn - The turn each building is constructed
##Row = Building
##Column = Turn

###A maximum of one building can be constructed each turn
for j=1:length(techs)
    @constraint(m, sum(building_turn[:,j]) <= 1)
end

###Each building must be constructed
for i=1:length(buildings)
    @constraint(m, sum(building_turn[i,:]) == 1)
end

###Each building is constructed the turn after its predecessor technology is researched
for i=1:length(buildings)
    for j in building_pred_dict[i]
        @constraint(m, pos*building_turn[i,:] == pos*final_tech_turn[j,:] + 1)
    end
end

##Building Status - The buildings that are active each turn
##Row = Building
##Column = Turn

###The status of each building is on after it has been constructed
for i=1:length(buildings)
    for j=1:length(techs)
        @constraint(m, building_status[i,j] >= building_turn[i,j])
    end
end

###The status of each building stays on for every turn after it has been constructed
for i=1:length(buildings)
    for j=1:(length(techs) - 1)
       @constraint(m, building_status[i,j] <= building_status[i,j+1])
    end
end
 
###The building is constructed one turn after its predecessor tech has been researched
for i=1:length(buildings)
    for j in building_pred_dict[i]
        @constraint(m, sum(building_status[i,:]) <= sum(tech_status[j,:]) - 1)
    end
end

@objective(m, Max, (sci_per_building[1]*building_status[1,:]'*final_tech_turn[:,:] + 
        sci_per_building[2]*building_status[2,:]'*final_tech_turn[:,:] + 
        sci_per_building[3]* building_status[3,:]'*final_tech_turn[:,:] .+ 2)*tech_sci_cost)

optimize!(m)
;

Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-04
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 8689 rows, 8316 columns and 51969 nonzeros
Model fingerprint: 0x4da5d08e
Model has 11907 quadratic objective terms
Variable types: 0 continuous, 8316 integer (8316 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e+02, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 839 rows and 823 columns
Presolve time: 0.17s
Presolved: 17336 rows, 16979 columns, 64176 nonzeros
Variable types: 0 continuous, 16979 integer (16896 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   35411    6.2107500e+05   6.700805e+03   0.000000e+00      5s
   49665    6.2107500e+05   0.000000e+00   0.000000e

## 5. Results and discussion ##

The result of our baseline model is the order to research technologies to minimize the turns to finish any given technology $x_k$. 

The output of the optimization program is a matrix of binary variables where each row represents a technology and the index of the 1 in that row represents the turn that technology is researched. 

The code below computes the order of the researched technologies in the original model without optimizing the buildings:

In [63]:
tech_seq = JuMP.value.(tech_turn);
tech_list = []
for i in 1:length(techs)
   for j in 1:length(techs)
       if tech_seq[j,i] == 1 
          push!(tech_list, tech_dict_rev[j])
       end
       continue
   end
end
println("Order of researching the technologies:")

show(stdout, "text/plain", tech_list)

Order of researching the technologies:
69-element Vector{Any}:
 :Mining
 :Wheel
 :AnimalHusbandry
 :Pottery
 :BronzeWorking
 :Masonry
 :IronWorking
 :Engineering
 :Machinery
 :Archery
 :Printing
 :HorsebackRiding
 :Writing
 :Construction
 :Stirrups
 :Currency
 :Apprenticeship
 :MilitaryEngineering
 :Gunpowder
 :MetalCasting
 :Ballistics
 :Castles
 :SiegeTactics
 :MilitaryScience
 :Rifling
 :Steel
 :Refining
 :Combustion
 :Plastics
 :SyntheticMaterials
 :Composites
 :Nanotechnology
 :CombinedArms
 :Mathematics
 :Education
 :Astronomy
 :ScientificTheory
 :Economics
 :ReplaceableParts
 :AdvancedBallistics
 :NuclearFission
 :Lasers
 :NuclearFusion
 :MilitaryTactics
 :Sailing
 :Shipbuilding
 :Sanitation
 :Buttress
 :Chemistry
 :Cartography
 :MassProduction
 :SquareRigging
 :Industrialization
 :SteamPower
 :Flight
 :Electricity
 :Radio
 :Rocketry
 :Computers
 :AdvancedFlight
 :Satellites
 :GuidanceSystems
 :Robotics
 :Astrology
 :StealthTechnology
 :Telecommunications
 :CelestialNavigation
 

From the baseline model, we observe that the last technology needed to achieve scientific victory is ``Robotics``. Therefore, the last six technologies are not necessary to research for, so we can simply remove them from our subsequent analysis. 

The code below computes the order of the researched technologiies in the new model **with** optimizing the buildings:

In [50]:
tech_seq = JuMP.value.(final_tech_turn);
final_tech_list = []
for i in 1:length(techs)
   for j in 1:length(techs)
       if tech_seq[j,i] == 1 
          push!(final_tech_list, tech_dict_rev[j])
       end
       continue
   end
end
println("Order of researching the technologies:")

show(stdout, "text/plain", final_tech_list)

Order of researching the technologies:
63-element Vector{Any}:
 :Pottery
 :Writing
 :AnimalHusbandry
 :Archery
 :Currency
 :Mining
 :HorsebackRiding
 :Apprenticeship
 :Mathematics
 :Education
 :Masonry
 :BronzeWorking
 :Astronomy
 :Construction
 :MilitaryEngineering
 :Stirrups
 :ScientificTheory
 :Gunpowder
 :MetalCasting
 :Economics
 :Sanitation
 :ReplaceableParts
 :Chemistry
 :Wheel
 :Sailing
 :Engineering
 :IronWorking
 :MilitaryTactics
 :Shipbuilding
 :Machinery
 :Castles
 :Buttress
 :SiegeTactics
 :MassProduction
 :Cartography
 :SquareRigging
 :Industrialization
 :Printing
 :MilitaryScience
 :Ballistics
 :SteamPower
 :Rifling
 :Refining
 :Flight
 :Steel
 :Electricity
 :Combustion
 :Radio
 :AdvancedFlight
 :Rocketry
 :Plastics
 :CombinedArms
 :AdvancedBallistics
 :SyntheticMaterials
 :NuclearFission
 :Composites
 :Lasers
 :Computers
 :GuidanceSystems
 :Nanotechnology
 :NuclearFusion
 :Satellites
 :Robotics

In [64]:
findall(x->x==:Writing,tech_list)[1]

13

In [65]:
findall(x->x==:Education,tech_list)[1]

35

In [66]:
findall(x->x==:Chemistry,tech_list)[1]

49

In [54]:
findall(x->x==:Writing,final_tech_list)[1]

2

In [55]:
findall(x->x==:Education,final_tech_list)[1]

10

In [56]:
findall(x->x==:Chemistry,final_tech_list)[1]

23

The results above compute the turns at which ``Writing``, ``Education`` and ``Chemistry`` are researched. All these three technologies are prerequisites to the `buildings`. It is evident that if we include the ``buildings`` to boost ``science`` yields, these three technologies are researched much earlier than the previous baseline model.

In [67]:
turns = 0
for i in 1:63
   turns += ceil(df_techs[tech_dict[tech_list[i]],3]/2)
end
turns

23896.0

In [68]:
turns = 0

for i in 1:findall(x->x==:Writing,tech_list)[1]
   turns += ceil(df_techs[tech_dict[tech_list[i]],3]/2)
end

for i in (findall(x->x==:Writing,tech_list)[1]+1):findall(x->x==:Education,tech_list)[1]
   turns += ceil(df_techs[tech_dict[tech_list[i]],3]/4)
end

for i in (findall(x->x==:Education,tech_list)[1]+1):findall(x->x==:Chemistry,tech_list)[1]
   turns += ceil(df_techs[tech_dict[tech_list[i]],3]/8)
end

for i in (findall(x->x==:Chemistry,tech_list)[1]+1):63
   turns += ceil(df_techs[tech_dict[tech_list[i]],3]/13)
end 

turns

7909.0

In [49]:
sum(ceil.(tech_sci_cost' ./ (sci_per_building[1]*JuMP.value.(building_status)[1,:]'*JuMP.value.(final_tech_turn[:,:]) 
            + sci_per_building[2]*JuMP.value.(building_status)[2,:]'*JuMP.value.(final_tech_turn[:,:])
            +  sci_per_building[3]*JuMP.value.(building_status)[3,:]'*JuMP.value.(final_tech_turn[:,:]) 
            .+ 2)))

4151.0

In the original model, we only consider technologies without any ``buildings`` that boost science. The result shows that it takes **23896** turns to reach scientific victory. However, after including ``buildings``, the new model suggests that it only takes **7909** turns (this number varies because the baseline model has many optimal solutions) to reach victory, which is significantly smaller than the previous number. Furthermore, we improve the model by optimizing the order of researched technologies based on the ``buildings``. The final result suggests that the number of required turns is further reduced to **4151**. 

## 6. Conclusion ##

**Civilization 6** is a critically acclaimed strategic game with significant popularity. The goal of each individual player is to compete with each other and achieve one of the victories (domination, scientific, cultural, diplomatic) at the earliest possible turn. Due to the scope and complexity of the game itself, few studies have used a *quantitative* approach to look into the game structure and give players advice on possible strategies they could employ in order to maximize their chance of winning. In this project, we focus on the scientific victory path only, identify the structure of the technology tree, and use mathematical modelling to characterize and solve the optimization problem. 

In our baseline model, we only consider ``science`` as a relevant factor, and we have successfully found the ideal path to achieve scientific victory as quickly as possible. However, when we add the ``buildings`` that could boost the ``science`` yields per turn, the total number of turns has significantly decreased. This result matches our intuition, and suggests that our optimization modelling successfully captures the effect of ``buildings`` on reducing **duration** of the victory path.

Moreover, by incorporating ``buildings`` that yield additional ``science`` in the subsequent turns, our model suggests that the **order** of researched technologies in the new victory path is vastly different from the order in our baseline result. In the simple setting, each player would be *myopic* and tends to research for *cheaper* technologies. However, when ``buildings`` are also considered, instead, each player's strategy is to become more *foresighted*, take advantage of the additional ``science`` yields, and research the technologies required for the ``buildings`` first, which might not necessarily be the cheapest. 

Our model has made some simplifying assumptions that make the actual game fit better to our optimization framework. In the future, one promising direction is to expand our current framework and consider additional factors that influence the ``science`` yields, such as the factor of ``production`` to construct buildings, the type of civilization each player chooses, and building a new city with additional ``buildings``. Furthermore, given that Civilization 6 is a multiplayer game, another possible direction is to incorporate actions of the other players into the optimization framework, such as establishing ``trade`` routes, or forming ``research agreements`` that are *mutually beneficial* in the short run. We look forward to reviewing future work with efficient optimization modelling that resembles the real game more closely. 

## 7. Author Contributions


#### 1. Modelling  
Aksel: 25%

Cameron: 25% 

Lingyun: 25% 

Prajwal: 25%

  
#### 2. Analysis  
Aksel: 25%

Cameron: 25%

Lingyun: 25%

Prajwal: 25%


#### 3. Data Gathering  
Aksel: 0%

Cameron: 100%  

Lingyun: 0% 

Prajwal: 0%


#### 4. Software Implementation  
Aksel: 17%

Cameron: 50%

Lingyun: 17% 

Prajwal: 16%


#### 5. Report writing   
Aksel: 25%

Cameron: 25%

Lingyun: 25% 

Prajwal: 25%